In [ ]:
import glob
import os
import json
from collections import Counter

In [ ]:
pagefiles = sorted(glob.glob('pages/*'))

In [ ]:
hopdata = []
for pagefile in pagefiles:
    with open(pagefile, 'r',encoding='utf8') as f:
        #text = f.readlines()
        text = f.read().splitlines();
    data = dict()
    name = os.path.splitext(os.path.basename(pagefile))[0].title().replace('_',' ')
    data['name'] = name

    #Files have multiple layout types for the PDF generation. Gross.
    cursor_idx = 2 #Keep track of where we are in file.
    if text[cursor_idx] == '':
        id_line = text[1]
        if 'BLEND' in id_line:
            data['is_blend'] = True

        else:
            data['hop_id'] = id_line
        cursor_idx = 3
        #some pages have additional empty lines
        while text[cursor_idx] == '':
            cursor_idx+=1
    else:
        cursor_idx = 1
    
    #Scan and build the description which is separated into multiple lines
    #print(cursor_idx)

    d = text[cursor_idx]
    while text[cursor_idx] != '':
        cursor_idx+=1
        d+=' ' + text[cursor_idx]

    data['description']=d.strip()
    
    #We now have tabular data of a column of lists.  Format is akin to:
    #
    # COUNTRY               AROMA PROFILE
    # NEW                   ORANGE • TEA • SAP
    # ZEALAND               PINE
    #
    #The aroma profile contians one or more items, one or more rows,
    # and the country is inexplicably columnar per word?  Love PDFs.
    #cursor_idx+=2
    while text[cursor_idx].split(' ')[0] != 'COUNTRY':
        cursor_idx+=1
    cursor_idx+=1
    
    country = ''
    aroma_profile = []
    
    while text[cursor_idx] != 'BEER STYLES':

        text[cursor_idx]
        if len(text[cursor_idx]) > 0:
            if text[cursor_idx][0] != ' ':
                split = text[cursor_idx].split()
                country += ' ' + split[0]
                aromas = ' '.join(split[1:])
                #if '•' in aromas:
                aroma_profile += aromas.split('•')
            else: #check for styles only (blank country col)
                #if '•' in text[cursor_idx]:
                aroma_profile += text[cursor_idx].split('•')
        cursor_idx +=1
        
    data['country'] = country.strip().title()
    data['aroma_profile'] = sorted([aroma.strip().title() for aroma in aroma_profile if aroma != ''])
        
   
    #Recommended beer styles
    # Follows dot separation as above, one or more lines.
    styles = []
    cursor_idx+=1
    while text[cursor_idx] != '':
        styles += text[cursor_idx].split('•')
        cursor_idx+=1
    data['styles'] = sorted([style.strip().title() for style in styles])

    
    """
    This section gets increasingly irregular in tabular format based on data available
    and becomes a rabbit hole of mess.  Most are low-high ranges, some are ~value, some
    are missing/omitted - this will be debugging snarl to get data that may or may not
    have real use.  Converting low/high to avg + range might be less pain to make a
    uniform compatible interface, but def some more debugging required here.
    
    
    #Skip to acid/oil section
    while text[cursor_idx].split(' ')[0] != 'ACID/OIL':
        cursor_idx+=1
    cursor_idx+=2
    alpha_acids_co_humulone = text[cursor_idx].replace('(',' ').replace(')',' ')
    alpha_acids_co_humulone = alpha_acids_co_humulone.replace('%',' ').split()
    print(alpha_acids_co_humulone)
    print(data['name'])
    try:
        alpha_acids_low = float(alpha_acids_co_humulone[2])
        alpha_acids_high = float(alpha_acids_co_humulone[4])
        data['alpha_acid_cohumulone_percent_low'] = alpha_acids_low
        data['alpha_acid_cohumulone_percent_high'] =alpha_acids_high
    except IndexError:
        pass
    
    
    #% alpha acids
    cursor_idx+=1
    alpha_acid = text[cursor_idx].replace('%',' ').split()
    data['alpha_acid_percent_low'] = float(alpha_acid[0])
    data['alpha_acid_percent_high'] = float(alpha_acid[1])
    
    #% beta acids
    cursor_idx +=4
    beta_acid = text[cursor_idx].replace('%',' ').split()
    data['beta_acid_percent_low'] = float(beta_acid[0])
    data['beta_acid_percent_high'] = float(beta_acid[1])
    
    #total oil
    cursor_idx +=4
    total_oil = text[cursor_idx].replace('%',' ').split()
    print(total_oil)
    data['total_oil_low'] = float(total_oil[0])
    if len(total_oil) == 2:
        print("total_oil : {}".format(total_oil))
        try:
            data['total_oil_high'] = float(total_oil[1][:-7])
            data['total_oil_units'] = total_oil[1][-7:]
        except ValueError:  #Case where only one unit is listed and not range
            data['total_oil_high'] = data['total_oil_low']
            data['total_oil_units'] = total_oil[1]
    else:
        data['total_oil_high'] = float(total_oil[1])
        data['total_oil_units'] = total_oil[2]
        
    #oil composition
    cursor_idx +=6
    
    for i in range(9):
        tokens =  text[cursor_idx + i ].replace('%',' ').split()
        print(tokens)
        if len(tokens) > 0:  #no data in pdf table if this fails
            oil_name = tokens[0].lower();
            if len(tokens) > 1:
                oil_percentage_low = tokens[1]
                oil_percentage_high = tokens[3]
            else:
                oil_percentage_low = 0
                oil_percentage_high = 0
            data[oil_name] = [oil_percentage_low,oil_percentage_high]
        else:
            break
            
    print(data)
    """
    
    #TODO: Mapping of styles and flavor notes to replace bad ones
    # such as PINEY --> PINE
    # and some crosswalk table between yakima chief beer styles and cloudburst?
    
    hopdata.append(data)

In [ ]:
#Tabulate some statistics on styles and aromas and print them.

styles = []
aromas = []
countries = []

#String replacement rules to reduce numbers of rare aroma categories
aroma_replacement_rules = {'Piney':'Pine',
    'Woody':'Wood',
    'Fruity':'Fruit',
    'Grassy':'Grass',
    'Fresh Cut Grass':'Grass',
    'Lemon-':'Lemon',
    'Floral Bouquet':'Floral',
    'Clean Citrus':'Citrus',
    'Resinous':'Resin',
    'Spice':'Spicy',
    'Sugar':'Sweet',
    'Pine Needle':'Pine',
    }

for hop in hopdata:
    for i,aroma in enumerate(hop['aroma_profile']):
        for k,v in aroma_replacement_rules.items():
            if aroma == k:
                hop['aroma_profile'][i] = v

    
    countries.append(hop['country'])
    aromas+=hop['aroma_profile']
    styles+=hop['styles']
    
countries = sorted(countries)
aromas = sorted(aromas)
styles = sorted(styles)

c = json.dumps(Counter(countries), indent = 2)
a = json.dumps(Counter(aromas), indent = 2)
s = json.dumps(Counter(styles), indent = 2)
print(c)
print(a)
print(s)

In [ ]:
b = json.dumps({'hops' : hopdata}, indent = 2, ensure_ascii=False)
print(b)
with open('yakimachiefhopdata.json', 'w', encoding='utf8') as f:
    json.dump({'hops' : hopdata}, f, indent = 2)
